In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



## Öffnen von Hdf mittels pandas

In [13]:
hdf = pd.HDFStore('../../data/raw/TestMessungen_NEU.hdf')
df = hdf.get("/x1/t1/trx_3_1")
df.head()

,index,Timestamp,trx_1_3_ifft_0,trx_1_3_ifft_1,trx_1_3_ifft_2,trx_1_3_ifft_3,trx_1_3_ifft_4,trx_1_3_ifft_5,trx_1_3_ifft_6,trx_1_3_ifft_7,...,trx_4_3_ifft_1992,trx_4_3_ifft_1993,trx_4_3_ifft_1994,trx_4_3_ifft_1995,trx_4_3_ifft_1996,trx_4_3_ifft_1997,trx_4_3_ifft_1998,trx_4_3_ifft_1999,trx_4_3_ts,target
0,0,1.492291e+09,0.017273,0.016833,0.016378,0.015908,0.015426,0.014929,0.014421,0.013900,...,0.032345,0.032998,0.033611,0.034182,0.034712,0.035198,0.035641,0.036040,1.488963e+12,"Empty_0.0,0.0_0.0,0.0"
1,1,1.492291e+09,0.004969,0.004496,0.004030,0.003580,0.003155,0.002772,0.002456,0.002239,...,0.015848,0.017529,0.019332,0.021220,0.023167,0.025153,0.027164,0.029188,1.488963e+12,"Empty_0.0,0.0_0.0,0.0"
2,2,1.492291e+09,0.016446,0.015995,0.015530,0.015051,0.014560,0.014056,0.013540,0.013014,...,0.016996,0.018922,0.020831,0.022722,0.024592,0.026439,0.028262,0.030058,1.488963e+12,"Empty_0.0,0.0_0.0,0.0"
3,3,1.492291e+09,0.018287,0.017823,0.017345,0.016855,0.016352,0.015838,0.015313,0.014778,...,0.051990,0.050347,0.048673,0.046971,0.045241,0.043486,0.041708,0.039909,1.488963e+12,"Empty_0.0,0.0_0.0,0.0"
4,4,1.492291e+09,0.013966,0.013449,0.012921,0.012382,0.011834,0.011277,0.010711,0.010139,...,0.016158,0.017800,0.019462,0.021136,0.022811,0.024482,0.026143,0.027791,1.488963e+12,"Empty_0.0,0.0_0.0,0.0"


## Beispiel Erkenner

### datensätze vorbereiten

In [25]:
from evaluation import *
from filters import *
from utility import *
from features import *
from new_features import *

# generate datasets
tst = ['1','2','3']
tst_ds = []

for t in tst:

    #df_tst = hdf.get('/x1/t'+t+'/trx_3_1')
    df_tst = hdf.get('/x1/t'+t+'/trx_1_2')
    
    lst = df_tst.columns[df_tst.columns.str.contains('_ifft_')]
    
    df_tst,_ = distortion_filter(df_tst)
    
    groups = get_trx_groups(df_tst)
    
    df_cf_mean = reduce_dim_PCA(cf_mean_window(df_tst, window=3, column_key="ifft", label=None ).fillna(0))
    
    df_std = rf_grouped(df_tst, groups=groups, fn=rf_std_single, label='target')
    df_mean = rf_grouped(df_tst, groups=groups, fn=rf_mean_single)
    df_p2p = rf_grouped(df_tst, groups=groups, fn=rf_ptp_single) # added p2p feature
    df_kurt = rf_grouped(df_tst, groups=groups, fn=rf_kurtosis_single)
    
    df_all = pd.concat( [df_std, df_mean, df_p2p, df_kurt], axis=1 ) 
     
    df_all = cf_std_window(df_all, window=4, label='target')
    
    df_all = cf_diff(df_all, label='target')
    
    #df_all = reduce_dim_PCA(df_all.fillna(0), n_comps=10, label='target')

    #df_all = pd.concat( [df_all, df_cf_mean], axis=1)
    
    df_tst_sum = generate_class_label_presence(df_all, state_variable='target')
    
    
            
    # remove index column
    df_tst_sum = df_tst_sum[df_tst_sum.columns.values[~df_tst_sum.columns.str.contains('index')].tolist()]
    

    
    #df_all = reduce_dim_LDA(df_tst_sum, n_comps=1, label="target")
    


    
    print('Columns in Dataset:',t)
    print(df_tst_sum.columns)
    

    tst_ds.append(df_tst_sum.copy())
    

Columns in Dataset: 1
Index(['col_diff_col_std_rf_std_trx_1_2_ifft_0',
       'col_diff_col_std_rf_std_trx_2_1_ifft_0',
       'col_diff_col_std_rf_std_trx_3_1_ifft_0',
       'col_diff_col_std_rf_std_trx_3_2_ifft_0',
       'col_diff_col_std_rf_std_trx_4_1_ifft_0',
       'col_diff_col_std_rf_std_trx_4_2_ifft_0',
       'col_diff_col_std_rf_mean_trx_1_2_ifft_0',
       'col_diff_col_std_rf_mean_trx_2_1_ifft_0',
       'col_diff_col_std_rf_mean_trx_3_1_ifft_0',
       'col_diff_col_std_rf_mean_trx_3_2_ifft_0',
       'col_diff_col_std_rf_mean_trx_4_1_ifft_0',
       'col_diff_col_std_rf_mean_trx_4_2_ifft_0',
       'col_diff_col_std_rf_ptp_trx_1_2_ifft_0',
       'col_diff_col_std_rf_ptp_trx_2_1_ifft_0',
       'col_diff_col_std_rf_ptp_trx_3_1_ifft_0',
       'col_diff_col_std_rf_ptp_trx_3_2_ifft_0',
       'col_diff_col_std_rf_ptp_trx_4_1_ifft_0',
       'col_diff_col_std_rf_ptp_trx_4_2_ifft_0',
       'col_diff_col_std_rf_kurt_trx_1_2_ifft_0',
       'col_diff_col_std_rf_kurt_trx_2_1

### validierung hold-out

In [ ]:
# holdout validation
print(hold_out_val(tst_ds, target='target', include_self=False, cl='rf', verbose=False, random_state=1))
print(hold_out_val(tst_ds, target='target', include_self=False, cl='ab', verbose=False, random_state=1))
print(hold_out_val(tst_ds, target='target', include_self=False, cl='nb', verbose=False, random_state=1))
print(hold_out_val(tst_ds, target='target', include_self=False, cl='knn', verbose=False, random_state=1))
print(hold_out_val(tst_ds, target='target', include_self=False, cl='ld', verbose=False, random_state=1))
print(hold_out_val(tst_ds, target='target', include_self=False, cl='qd', verbose=False, random_state=1))

### extra feature "peak to peak"

In [ ]:
# generate datasets
tst = ['1','2','3']
tst_ds_2 = []

for t in tst:

    df_tst = hdf.get('/x1/t'+t+'/trx_3_1')
    lst = df_tst.columns[df_tst.columns.str.contains('_ifft_')]
    
    #df_tst_cl,_ = distortion_filter(df_tst_cl)
    
    groups = get_trx_groups(df_tst)
    df_std = rf_grouped(df_tst, groups=groups, fn=rf_std_single, label='target')
    df_mean = rf_grouped(df_tst, groups=groups, fn=rf_mean_single)
    df_p2p = rf_grouped(df_tst, groups=groups, fn=rf_ptp_single)
    
    df_all = pd.concat( [df_std, df_mean, df_p2p], axis=1 )
    
    df_all = cf_std_window(df_all, window=4, label='target')
    
    df_tst_sum = generate_class_label_presence(df_all, state_variable='target')
    
    # remove index column
    df_tst_sum = df_tst_sum[df_tst_sum.columns.values[~df_tst_sum.columns.str.contains('index')].tolist()]
    print('Columns in Dataset:',t)
    print(df_tst_sum.columns)
    

    tst_ds_2.append(df_tst_sum.copy())
    

In [ ]:
# holdout validation
print(hold_out_val(tst_ds_2, target='target', include_self=False, cl='rf', verbose=False, random_state=1))

## Schließen von HDF Store

In [ ]:
hdf.close()